#Problem Description 

> **You have been asked to help a candidate to become US president. The winner  of  the election will be the candidate winning the most grand electors.**

> **Grand electors are attributed at the state level: in each of the 51 states, there is a given number of grand electors to win (roughly, but not exactly, proportional to the size of the state) and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.**

> **Because the number of grand electors is not exactly proportional to the population, some states can be prioritized to increase the return on investment of the campaign. We assume here there are only 2 candidates, and no history (no trend of certain states to vote for a particular candidate or party). Hence, each vote is equally "expensive" to get, but some states grant more grand elector per capita.**

> **You are provided with 2 tables: one giving the number of Grand Electors per state, the other the population per state.**

> **You are asked to identify the states that should be prioritized to win the election, with a smart but simple algorithm (brute force computation of all possibilities to find the absolute optimum is not accepted, as it would be to computationally expensive). It is ok not to find the overall optimum, we just want a strategy that makes sense.**

> **(This is of course a very simplistic view of reality, but it is a nice starting point to play with data and analyze possibilities).**

> **First take a few minutes to think about what you need to do. Steps will be suggested hereafter, but a big part of the data scientist's job is to identify the flow of operations when being asked a practical question, so it is important you start exercising on that in addition to programming.**

> **Here is what we are suggesting to do: we will rank states by decreasing number of grand electors per capita. The first states in the list will be the most valuable (you get a large number of grand electors by convincing a small number of people to vote for you). We will target all the states at the top of the list until the cumulative sum (also called running total) of grand electors won is larger than half the total number of Grand Electors in the country.**

> **To do that, we need (you are allowed to create as many intermediary tables as you want, to keep queries short):**

> **
To join the 2 tables:

    A. You notice States are not capitalized the same way in both tables (one is in uppercase letters, the other not), so you will first need to convert all to uppercase, for instance.
    B. Now you can join the tables on the state key.

  1. Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.
  2. To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.
  3. To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.
  4. To compute the running total of Grand Electors in that sorted list.
  Hint: you can get inspiration from here to compute a running total from here:  https://stackoverflow.com/questions/21382766/cumulative-summing-values-in-sqlite
  5. Independently, to compute the half of the total of Grand Electors overall (in the whole country):
  6. This is the threshold we need to reach for winning the presidential election.
  7. To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.
  Hint: You can do that in 2 steps:
  8. Select all the states for which the running total is below or equal to the threshold.
  9. Add the first state for which the running total is larger than the threshold.
  10. Can you draw some conclusions from the result? Is it in line with your expectations? How many states do you end up with in the target list? Is it a small or a large number? Do you think it would be a good recommendation to target those states?

#1.0 CONNECTING TO DATABASE AND LOADING DATA FROM CSV FILES

##1.2 IMPORTING DATA FROM CSV FILES

In [1]:
# load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database

%sql sqlite://

'Connected: @None'

In [1]:
# importing python modules
import csv # for reading csv files
import pandas as pd # for manipulating data from our csv files

In [5]:
# load and drop our data into sqlite table

# DATA 1 : GrandElectors by state
with open("GrandElectors_by_state.csv","r") as G:
  GrandElectors_by_state = pd.read_csv(G, index_col=0,encoding="utf-8")

# drop to sql
%sql DROP TABLE if EXISTS GrandElectors_by_state;
%sql PERSIST GrandElectors_by_state;

 * sqlite://
Done.
 * sqlite://


'Persisted grandelectors_by_state'

In [6]:
# load and drop our data into sqlite table

# DATA 2 : Population by state
with open("Population_by_state.csv","r") as G:
  Population_by_state = pd.read_csv(G, index_col=0,encoding="utf-8")

# drop to sql
%sql DROP TABLE if EXISTS Population_by_state;
%sql PERSIST Population_by_state;

 * sqlite://
Done.
 * sqlite://


'Persisted population_by_state'

In [7]:
# DISPLAY SAMPLE OF DATA 1: 
%%sql
SELECT * FROM GrandElectors_by_state LIMIT 20

 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55
Colorado,9
Connecticut,7
District of Columbia,3
Delaware,3
Florida,29


In [8]:
# DISPLAY SAMPLE OF DATA 2: 
%%sql
SELECT * FROM Population_by_state LIMIT 20

 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995
PENNSYLVANIA,12802503
OHIO,11613423
GEORGIA,10214860
NORTH CAROLINA,10042802
MICHIGAN,9922576


#2.0 Working on our TABLES

____ TABLES and Columns ____
> GrandElectors_by_state
>> Columns : State	GrandElectors

> Population_by_state
>> Columns: State	Population

##2.1 Joining Tables



###2.1.0 Q1

> To join the 2 tables: 
  1. **You notice States are not capitalized the same way in both tables (one is in uppercase letters, the other not), so you will first need to convert all to uppercase, for instance.
Now you can join the tables on the state key.**

In [9]:
# a. Capitalizing states in GrandElectors_by_state
%%sql 
SELECT UPPER(State) FROM GrandElectors_by_state LIMIT 3

 * sqlite://
Done.


UPPER(State)
ALABAMA
ALASKA
ARIZONA


In [53]:
# b. joining both tables
%%sql
SELECT Population_by_state.State, Population,GrandElectors_by_state.GrandElectors
FROM Population_by_state
LEFT JOIN GrandElectors_by_state
ON (GrandElectors_by_state.State = Population_by_state.State) 
LIMIT 10

 * sqlite://
Done.


State,Population,GrandElectors
CALIFORNIA,39144818,None
TEXAS,27469114,None
FLORIDA,20271272,None
NEW YORK,19795791,None
ILLINOIS,12859995,None
PENNSYLVANIA,12802503,None
OHIO,11613423,None
GEORGIA,10214860,None
NORTH CAROLINA,10042802,None
MICHIGAN,9922576,None


In [36]:
%%sql
SELECT GrandElectors FROM GrandElectors_by_state LIMIT 10

 * sqlite://
Done.


GrandElectors
9
3
11
6
55
9
7
3
3
29


###2.1.1 Q2 
> Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that

In [49]:
# 
%%sql
SELECT State, REPLACE("State", "District of Columbia", "DC") 
FROM GrandElectors_by_state WHERE State="District of Columbia"

 * sqlite://
Done.


State,"REPLACE(""State"", ""District of Columbia"", ""DC"")"
District of Columbia,DC


###2.1.2 Q3 
> To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.

In [58]:
%%sql 
SELECT ROUND(AVG(GrandElectors)),ROUND(AVG(Population)) FROM GrandElectors_by_state
LEFT JOIN 


 * sqlite://
Done.


ROUND(AVG(GrandElectors))
11.0


###2.1.3 Q4 
> To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.

In [59]:
%%sql
SELECT ROUND(GrandElectors), State FROM GrandElectors_by_state
ORDER BY GrandElectors DESC

 * sqlite://
Done.


ROUND(GrandElectors),State
55.0,California
38.0,Texas
29.0,Florida
29.0,New York
20.0,Illinois
20.0,Pennsylvania
18.0,Ohio
16.0,Georgia
16.0,Michigan
15.0,North Carolina


###2.1.4 Q5 
> To compute the running total of Grand Electors in that sorted list.

In [64]:
#😪out of time

%%sql
SELECT
    Sum(GrandElectors)
FROM GrandElectors_by_state 

 * sqlite://
Done.


Sum(GrandElectors)
538
